Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situuation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [3]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [4]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [5]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 2.89 ms, sys: 2.15 ms, total: 5.04 ms
Wall time: 12.7 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [6]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://192.168.1.88/22793/1 Dashboard: http://192.168.1.88/22793/1:8787/status,Cluster Workers: 1 Cores: 6 Memory: 17.18 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [7]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 479 ms, sys: 42.1 ms, total: 521 ms
Wall time: 1.99 s


[1.3111611115161288,
 1.6710683157391781,
 1.7164428468748616,
 1.800622553076163,
 1.727166530465721,
 1.2435391897040307,
 1.7056947623387475,
 1.0596045673985244,
 1.8857891985999329,
 1.6158345289423508,
 1.919814401986857,
 1.395797021070106,
 1.2328607075558677,
 1.8716573039246027,
 1.1932916116625059,
 1.8603989952102773,
 1.0328788488139926,
 1.2028754763374359,
 1.1320963319137496,
 1.721696627844525,
 1.3847105665467672,
 1.5209424563772096,
 1.3011548414159444,
 1.0068324689443795,
 1.7988131312639295,
 1.623488584383071,
 1.3647231496854348,
 1.766826156334631,
 1.389170976744607,
 1.5207986950905719,
 1.4630442325144821,
 1.0928294302684436,
 1.158048714608169,
 1.5057672112122034,
 1.3051738493142957,
 1.7915686291889927,
 1.4641025730591553,
 1.5689841130675308,
 1.8939205181190344,
 1.4897614212992683,
 1.837505474832139,
 1.6853162616031758,
 1.5703147725335822,
 1.324593341124221,
 1.9752197685334623,
 1.9836642244658802,
 1.1166666932969158,
 1.4426310014217711,
 1.

2: Use async/await to handle each locally
-----------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [8]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [9]:
client.sync(run_all, filenames)

[1.357297484443608,
 1.459251410204741,
 1.017850718204474,
 1.0103207961577314,
 1.930213720287629,
 1.2857328191406316,
 1.5820068867733315,
 1.235126464814356,
 1.3017973825770213,
 1.280494353530698,
 1.2621753092912116,
 1.0692221690014656,
 1.3409450020077975,
 1.76095194793966,
 1.406913349808808,
 1.1629781372157746,
 1.284890053291389,
 1.296110865907317,
 1.6043190271860084,
 1.6183468236638476,
 1.0371823069625306,
 1.1440474354383736,
 1.2700546868202787,
 1.4251602095333846,
 1.8081116247079962,
 1.3095577060322032,
 1.378127448760599,
 1.4198662907544803,
 1.289672077527427,
 1.3311575824239048,
 1.653425104334226,
 1.3210495246312448,
 1.4387470035896208,
 1.1943843706156694,
 1.3371702554982288,
 1.402034603527444,
 1.986081438672693,
 1.727300297178989,
 1.0947838765328108]

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [10]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 505 ms, sys: 44.8 ms, total: 550 ms
Wall time: 2.44 s
